In [25]:
#from tiingo import TiingoClient
import pandas as pd
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross, SignalStrategy, plot_heatmaps
from backtesting.test import SMA, GOOG
import requests

In [2]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=240&since=1522030972') 

In [3]:
resp.json()
dat = resp.json()['result']
data = {}
#Using this to format the data into a pandas dataframe

In [4]:
for i in range(len(dat['XXBTZUSD'])):
    data[dat['XXBTZUSD'][i][0]] = dat['XXBTZUSD'][i][1:]


In [5]:
dates = [datetime.fromtimestamp(key) for key in data.keys()]

In [6]:
ohlc_data = [value[:6] for value in data.values()]

In [7]:
ohlc_dict = {'Open': [float(val[0]) for val in ohlc_data],
             'High': [float(val[1]) for val in ohlc_data],
             'Low': [float(val[2]) for val in ohlc_data],
             'Close': [float(val[3]) for val in ohlc_data],
             'Volume': [float(val[5]) for val in ohlc_data]}

In [8]:
df = pd.DataFrame(ohlc_dict, index=dates)

In [9]:
df = (df / 1e6).assign(Volume=df.Volume * 1e6)
#Because backtesting.py is made for stocks, It doesn't support buying incremental values of BTC so to get around that we're just using price per Sat

In [11]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

#Unused for the cloud strategy, just using as reference

In [10]:
def maxminIch(n1, n2):
    line = (n1+n2)/2
    return line

These functions are calculating the average of period highs/lows for each line. 
I decided to use multiple functions over a single one (as you can see they all do the same thing) simply for better readability in the strat definition.

In [12]:
def Tenkan(n1, n2):
    line = (n1+n2)/2
    return line

In [13]:
def Kijun(n1, n2):
    line = (n1+n2)/2
    return line

In [14]:
def addshift(s1, window):
    shifted = s1.shift(window)
    return shifted

In [15]:
def senkouA(s1, window):
    shifted = s1.shift(window).fillna(0)
    return shifted

In [16]:
def senkouB(s1, window):
    shifted = s1.shift(window).fillna(0)
    return shifted

In [17]:
t = maxminIch(df['High'].rolling(20).max(), df['Low'].rolling(20).min())
k = maxminIch(df['High'].rolling(60).max(), df['Low'].rolling(60).min())
senkouB(((df['High'].rolling(20).max() + df['Low'].rolling(20).min()) / 2), 30)
#((df['High'].rolling(20).max() + df['Low'].rolling(20).min()) / 2)

#T and K are rolling averages of the period high and lows

2024-03-18 15:00:00    0.000000
2024-03-18 19:00:00    0.000000
2024-03-18 23:00:00    0.000000
2024-03-19 03:00:00    0.000000
2024-03-19 07:00:00    0.000000
                         ...   
2024-07-15 19:00:00    0.056888
2024-07-15 23:00:00    0.056888
2024-07-16 03:00:00    0.057214
2024-07-16 07:00:00    0.057227
2024-07-16 11:00:00    0.057227
Length: 720, dtype: float64

In [18]:
class IchimokuCloud(Strategy): #This should be a signal strategy - signals equal to actionable cloud patterns, set entry then pass to set signal
    nt = 20
    nk = 60
    nsb = 120
    nc = 30
    
    def init(self):
        close=self.data.Close
        self.tenkan = self.I(Tenkan, self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min())
        self.kijun = self.I(Kijun, self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min())
        self.senkouA = self.I(senkouA,
                              (maxminIch(self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min()) + maxminIch(self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min()))/2,
                              self.nc)
        self.senkouB = self.I(senkouB, maxminIch(self.data['High'].s.rolling(self.nsb).max(), self.data['Low'].s.rolling(self.nsb).min()), self.nc)
        self.lagging = self.I(addshift, close.s, (self.nc * -1))
        
    def next(self): #Kumo twist: If Senkou B > Senkou A and next-price > SB, Buy
        if (crossover(self.data.Close, self.senkouB) and self.senkouB > self.senkouA): 
            self.buy()
        elif (cross(self.tenkan, self.kijun) and self.kijun > self.tenkan):
            self.sell()

These frames are the cloud strategy. I haven't delved too deep in Backtesting.py so they're fairly simple. It probably wouldn't be difficult to flesh these out into a full strategy. Definitley keep model overfitting in your mind when testing strategies. Here's a great resource to read about it if you're interested: https://www.davidhbailey.com/dhbtalks/battle-quants.pdf

In [19]:
class IchimokuSignals(SignalStrategy):
    nt = 20
    nk = 60
    nsb = 120
    nc = 30
    
    def init(self):
        super().init()
        #precompute cloud
        close=self.data.Close
        self.tenkan = self.I(Tenkan, self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min())
        self.kijun = self.I(Kijun, self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min())
        self.senkouA = self.I(senkouA,
                              (maxminIch(self.data['High'].s.rolling(self.nt).max(), self.data['Low'].s.rolling(self.nt).min()) + maxminIch(self.data['High'].s.rolling(self.nk).max(), self.data['Low'].s.rolling(self.nk).min()))/2,
                              self.nc)
        self.senkouB = self.I(senkouB, maxminIch(self.data['High'].s.rolling(self.nsb).max(), self.data['Low'].s.rolling(self.nsb).min()), self.nc)
        self.lagging = self.I(addshift, close.s, (self.nc * -1))
        #establish signal points:
        signal = crossover(self.data.Close, self.senkouB) and self.senkouB > self.senkouA #Simple cloud cross strategy
        signal = signal

In [20]:
bt = Backtest(df, IchimokuCloud,
              cash=100, commission=.002,
              exclusive_orders=True, margin=0.5)

This function starts the backtest, values can be anything you want. Exclusive orders just means you wont open new positions if you're already in one.

In [21]:
output = bt.run()
bt.plot()

/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1339', ...)

In [22]:
output['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-3027,93,150,0.066054,0.066862,-2.445126,-0.012229,2024-04-03 03:00:00,2024-04-12 15:00:00,9 days 12:00:00
1,-2923,150,210,0.066728,0.066589,0.406522,0.002084,2024-04-12 15:00:00,2024-04-22 15:00:00,10 days 00:00:00
2,2936,210,215,0.066722,0.066423,-0.877210,-0.004478,2024-04-22 15:00:00,2024-04-23 11:00:00,0 days 20:00:00
3,2917,215,248,0.066556,0.062280,-12.472352,-0.064243,2024-04-23 11:00:00,2024-04-28 23:00:00,5 days 12:00:00
4,-2722,248,291,0.062156,0.064559,-6.540858,-0.038660,2024-04-28 23:00:00,2024-05-06 03:00:00,7 days 04:00:00
5,2413,291,297,0.064688,0.063682,-2.426072,-0.015543,2024-05-06 03:00:00,2024-05-07 03:00:00,1 days 00:00:00
6,2370,297,299,0.063810,0.063711,-0.234546,-0.001551,2024-05-07 03:00:00,2024-05-07 11:00:00,0 days 08:00:00
7,2362,299,311,0.063838,0.062144,-4.000334,-0.026530,2024-05-07 11:00:00,2024-05-09 11:00:00,2 days 00:00:00
8,2293,311,329,0.062269,0.061400,-1.991904,-0.013951,2024-05-09 11:00:00,2024-05-12 11:00:00,3 days 00:00:00
9,-2265,329,344,0.061277,0.061924,-1.464549,-0.010552,2024-05-12 11:00:00,2024-05-14 23:00:00,2 days 12:00:00


In [23]:
output

Start                     2024-03-18 15:00:00
End                       2024-07-16 11:00:00
Duration                    119 days 20:00:00
Exposure Time [%]                   87.083333
Equity Final [$]                    118.35111
Equity Peak [$]                     118.35111
Return [%]                           18.35111
Buy & Hold Return [%]               -4.150444
Return (Ann.) [%]                   66.236531
Volatility (Ann.) [%]              174.603728
Sharpe Ratio                         0.379353
Sortino Ratio                        1.287542
Calmar Ratio                         1.503059
Max. Drawdown [%]                  -44.067816
Avg. Drawdown [%]                  -10.959163
Max. Drawdown Duration       78 days 16:00:00
Avg. Drawdown Duration       11 days 12:00:00
# Trades                                   16
Win Rate [%]                             37.5
Best Trade [%]                      11.951497
Worst Trade [%]                     -6.424289
Avg. Trade [%]                    

In [28]:
output, heatmap = bt.optimize(nt=range(5, 30, 5),
                     nk=range(10, 70, 5),
                     nsb=range(50, 200, 5),
                     maximize='Equity Final [$]',
                     constraint=lambda param: param.nt < param.nk < param.nsb, return_heatmap=True)

plot_heatmaps(heatmap)

/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1450 configurations.
  output = _optimize_grid()
/Users/iowapsychopath/anaconda3/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/5 [00:00<?, ?it/s]

GridPlot(id='p1500', ...)